## Concrete Strength Prediction using Linear Regression

This notebook analyzes concrete strength data and builds a linear regression model to predict concrete strength based on various components

## Import Required Libraries

Import pandas, numpy, matplotlib, seaborn, and sklearn libraries for data manipulation, visualisation and machine learning.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings('ignore')

# set style for plots
plt.style.use('seaborn-v0_8')
sns.set_palette('husl')

print("Libraries imported successfully")

: 